In [16]:
import networkx as nx
import pandas as pd
import numpy as np
import arviz as az
import matplotlib.pyplot as plt
import pymc as pm
import scipy as sp
import scipy.stats
import seaborn as sns

from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import BDeuScore
from tqdm import tqdm
from itertools import combinations
from graphviz import Digraph
from itertools import combinations
from IPython.display import clear_output
from timeit import default_timer as timer
from datetime import timedelta

import cloudpickle
import os

In [9]:
networks = dict()
df_path = './cross_validated_datasets/lucas0/1000/'

for i, file in tqdm(enumerate([*filter(lambda x: 'lucas' in x, os.listdir(df_path))])):
    data = pd.read_csv(df_path + file)
    
    networks[i] = list()

    scoring_method = BDeuScore(data=data)    
    est = HillClimbSearch(data=data)
    networks[i].append(est.estimate(scoring_method=scoring_method, max_indegree=2, max_iter=int(1e4), show_progress=False))

100it [00:23,  4.27it/s]


In [10]:
data = pd.read_csv(df_path + 'lucas0_1000_0.csv')
edges = [*combinations(data.columns, 2)]

observed_networks = [[] for i in range(len(networks))]

for i, file in tqdm(enumerate([*filter(lambda x: 'lucas' in x, os.listdir(df_path))])):
    for j, network in enumerate(networks[i]):
        observed_networks[i].append([])
        
        for edge in edges:
            if (edge in network.edges):
                observed_networks[i][j].append(-1)
            elif ((edge[1], edge[0]) in network.edges):
                observed_networks[i][j].append(1)
            else: 
                observed_networks[i][j].append(0)

observed_networks = [np.array(x) for x in observed_networks]
observed_networks = np.array(observed_networks)

100it [00:00, 34094.49it/s]


In [11]:
values = [-1, 0, 1]
k = len(values)          
n = len(networks)
total_count = len(observed_networks[0])

In [19]:
def get_edge_frequency(values, observed_networks, edge_index):
    """
    values: list of possible values for each edge (ex: [0, 1, 2] or [-1, 0, 1])
    observed_networks: list of networks observed
    edge_index: edged which will be counted
    """
    
    observations = list()
    
    for edge_list in observed_networks:
        edges_obs = edge_list[:, edge_index]

        rightEdge = np.sum(edges_obs == values[0])
        noEdge = np.sum(edges_obs == values[1])
        leftEdge = np.sum(edges_obs == values[2])

        observations.append([rightEdge, noEdge, leftEdge])
    
    return observations


def learn_edge(edge, edge_index, chains=10, tune=10000, draws=10000, nthreads=10):
    observed_edges = get_edge_frequency(values, observed_networks, edge_index)

    with pm.Model() as model_dm_explicit:
        frac = pm.Dirichlet("frac", a=np.ones(k))
        conc = pm.Lognormal("conc", mu=1, sigma=1)
        counts = pm.DirichletMultinomial(
            "counts", n=total_count, a=frac * conc, shape=(n, k), observed=observed_edges
        )

        trace_dm_explicit = pm.sample(chains=chains, tune=tune, draws=draws, step=pm.NUTS(), return_inferencedata=False, cores=nthreads)

        Model = cloudpickle.dumps({
                'model': model_dm_explicit, 
                'frac':  frac, 
                'conc': conc, 
                'counts': counts, 
                'trace': trace_dm_explicit
            })
    
        file = open('./mcmc/lucas0_10000' + str(chains) + '_t' + str(tune) + '_d'+ str(draws) + '_mcmc_' + edge[0]+"-"+edge[1] + '.pickle', 'wb')
        file.write(Model)
        
        return trace_dm_explicit

In [18]:
start = timer()

models = dict()

for i, edge in enumerate(tqdm(edges)):
    models[edge[0]+"-"+edge[1]] = learn_edge(edge, i, chains=10, tune=2500, draws=2500, nthreads=10)
    
    clear_output(wait=True)

end = timer()
elapsed_time = timedelta(seconds=end-start)
print(elapsed_time)

  0%|                                                    | 0/66 [00:00<?, ?it/s]Multiprocess sampling (10 chains in 10 jobs)
NUTS: [frac, conc]


Sampling 10 chains for 2_500 tune and 2_500 draw iterations (25_000 + 25_000 draws total) took 5 seconds.
  0%|                                                    | 0/66 [00:11<?, ?it/s]


ValueError: Can only convert xarray dataarray, xarray dataset, dict, netcdf filename, numpy array, pystan fit, emcee fit, pyro mcmc fit, numpyro mcmc fit, cmdstan fit csv filename, cmdstanpy fit to InferenceData, not MultiTrace